# Glaucoma Segmentation


## Imports

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import defaultdict
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

from networks import *
from training import *
from utils import *

## Setup

In [ ]:
IMAGE_DIR = '../data/ORIGA/Images_Cropped'
MASK_DIR = '../data/ORIGA/Masks_Cropped'
LOGS_DIR = '../logs/'
CHECKPOINT_DIR = '../checkpoints/'
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
IN_CHANNELS = 3
OUT_CHANNELS = 3
NETWORK_NAME = 'unet'
LOSS_FUNCTION = 'dice'
OPTIMIZER = 'adam'
ARCHITECTURE = 'multiclass'  # multiclass, binary, dual, cascade
SCHEDULER = 'plateau'
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 3
LAYERS = [32, 64, 128, 256, 512]
EARLY_STOPPING_PATIENCE = 10
LOG_INTERVAL = 5
SAVE_INTERVAL = 10
NUM_WORKERS = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PIN_MEMORY = True if DEVICE == 'cuda' else False
USE_WANDB = False
DEEP_SUPERVISION = False

os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f'''CONFIGURATION:
    PyTorch version: {torch.__version__}
    Using device: {DEVICE}
    Network: {NETWORK_NAME}
    Architecture: {ARCHITECTURE}
    Optimizer: {OPTIMIZER}
    Loss function: {LOSS_FUNCTION}
    Scheduler: {SCHEDULER}
    Image directory: {IMAGE_DIR}
    Mask directory: {MASK_DIR}
    Input image height & width: {IMAGE_HEIGHT}x{IMAGE_WIDTH}
    Number of input channels: {IN_CHANNELS}
    Number of output channels: {OUT_CHANNELS}
    Layers: {LAYERS}
    Batch size: {BATCH_SIZE}
    Learning rate: {LEARNING_RATE}
    Epochs: {EPOCHS}
    Early stopping patience: {EARLY_STOPPING_PATIENCE}
    Save interval: {SAVE_INTERVAL}
    Log interval: {LOG_INTERVAL}
    Number of workers: {NUM_WORKERS}
    Pin memory: {PIN_MEMORY}
    Weight & Biases: {USE_WANDB}
    Deep supervision: {DEEP_SUPERVISION}''')

## Dataset

In [ ]:
def polar_transform(img, **kwargs):
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    value = np.sqrt(((width / 2.0) ** 2.0) + ((height / 2.0) ** 2.0))
    polar_img = cv.linearPolar(img, center, value, cv.WARP_FILL_OUTLIERS)
    polar_img = cv.rotate(polar_img, cv.ROTATE_90_COUNTERCLOCKWISE)
    return polar_img


def inverse_polar_transform(img, **kwargs):
    img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    value = np.sqrt(((width / 2.0) ** 2.0) + ((height / 2.0) ** 2.0))
    cartesian_img = cv.linearPolar(img, center, value, cv.WARP_INVERSE_MAP | cv.WARP_FILL_OUTLIERS)
    return cartesian_img


def arctan(x):
    return 1e-7 + (1 - 2 * 1e-7) * (0.5 + torch.arctan(x) / torch.tensor(np.pi))


In [ ]:
example_ds = OrigaDataset(IMAGE_DIR, MASK_DIR, os.listdir(IMAGE_DIR)[:1], A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=1.0),  # rotate by 0, 90, 180, or 270 degrees
    # A.Rotate(limit=30, p=0.33, border_mode=cv.BORDER_CONSTANT),
    # A.Lambda(image=polar_transform, mask=polar_transform),
    # A.Lambda(image=inverse_polar_transform, mask=inverse_polar_transform),

    # A.CLAHE(p=0.5),
    # A.RandomBrightnessContrast(p=0.5),
    # A.GridDistortion(p=0.5, border_mode=cv.BORDER_CONSTANT),
    # A.MedianBlur(p=0.5),
    # A.RandomToneCurve(p=0.5),
    # A.MultiplicativeNoise(p=0.5),
    ToTensorV2(),
]))
example_loader = DataLoader(example_ds, batch_size=1, shuffle=True)

example_image, example_mask = next(iter(example_loader))
print(f'Image shape: {example_image.shape}')
print(f'Mask shape: {example_mask.shape}')

unique, counts = np.unique(example_mask, return_counts=True)
print(f'Unique values and their counts in mask: {dict(zip(unique, counts))}')

# Plot example augmented images and masks
fig, ax = plt.subplots(3, 6, figsize=(12, 6))
ax = ax.ravel()
for i in range(0, 3 * 6, 2):
    batch = next(iter(example_loader))
    images, masks = batch
    image, mask = images[0], masks[0]
    image = image.permute(1, 2, 0).numpy()
    mask = mask.numpy()
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i + 1].imshow(mask, cmap='gray')
    ax[i + 1].axis('off')
plt.show()

In [ ]:
train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=1.0),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    ToTensorV2(),
])

train_loader, val_loader, test_loader = load_origa(
    IMAGE_DIR, MASK_DIR, 0.7, 0.15, 0.15,
    train_transform, val_transform, val_transform, BATCH_SIZE, PIN_MEMORY, NUM_WORKERS
)


## Model

In [ ]:
model = get_network(NETWORK_NAME, in_channels=IN_CHANNELS, out_channels=OUT_CHANNELS, layers=LAYERS).to(DEVICE)
optimizer = get_optimizer(OPTIMIZER, model.parameters(), lr=LEARNING_RATE)
criterion = get_criterion(LOSS_FUNCTION, num_classes=OUT_CHANNELS)
scheduler = get_scheduler(SCHEDULER, optimizer, factor=0.1, patience=5, verbose=True)
scaler = None
# scaler = torch.cuda.amp.GradScaler()

init_weights(model)

## Training

In [ ]:
if ARCHITECTURE == 'multiclass':
    hist = train_multiclass(
        model, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
        save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
        log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
        save_best_model=True, plot_examples='none', show_plots=False,
    )

if ARCHITECTURE == 'binary':
    hist = train_binary(
        model, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
        save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
        log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
        save_best_model=True, plot_examples='none', show_plots=False,
    )

if ARCHITECTURE == 'dual':
    hist = train_dual(
        model, criterion, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
        save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
        log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
        save_best_model=True, plot_examples='none', show_plots=False,
    )

if ARCHITECTURE == 'cascade':
    binary_model = Unet(3, 1, LAYERS).to(DEVICE)
    checkpoint = torch.load(CHECKPOINT_DIR + 'binary-model.pth')
    binary_model.load_state_dict(checkpoint)
    hist = train_cascade(
        binary_model, model, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
        save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
        log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
        save_best_model=True, plot_examples='none', show_plots=False,
    )


In [ ]:
# Plot metrics
used_metrics = sorted([m[6:] for m in hist.keys() if m.startswith('train_')])
fig, ax = plt.subplots(4, 4, figsize=(14, 8))
ax = ax.ravel()

for i, metric in enumerate(used_metrics):
    ax[i].plot(hist[f'train_{metric}'], label=f'train')
    ax[i].plot(hist[f'val_{metric}'], label=f'val')
    ax[i].set_title(metric[0].upper() + metric[1:].replace('_', ' '))
    if metric != 'loss':
        ax[i].set_ylim(top=1)
    ax[i].legend()

for ax in ax[len(used_metrics):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

## Testing

In [ ]:
if ARCHITECTURE == 'multiclass':
    results = evaluate('multiclass', model, criterion, DEVICE, test_loader)

if ARCHITECTURE == 'binary':
    results = evaluate('binary', model, criterion, DEVICE, test_loader)

if ARCHITECTURE == 'dual':
    results = evaluate('dual', model, criterion, DEVICE, test_loader)

if ARCHITECTURE == 'cascade':
    binary_model = Unet(3, 1, LAYERS).to(DEVICE)
    checkpoint = torch.load(CHECKPOINT_DIR + 'binary-model.pth')
    binary_model.load_state_dict(checkpoint)
    results = evaluate('cascade', model, criterion, DEVICE, test_loader, model0=binary_model)

In [ ]:
if ARCHITECTURE == 'multiclass':
    plot_results_from_loader('multiclass', test_loader, model, DEVICE, n_samples=4,
                             save_path=f'{LOGS_DIR}/evaluation.png')

if ARCHITECTURE == 'binary':
    plot_results_from_loader('binary', test_loader, model, DEVICE, n_samples=4,
                             save_path=f'{LOGS_DIR}/evaluation.png')

if ARCHITECTURE == 'dual':
    plot_results_from_loader('dual', test_loader, model, DEVICE,
                             n_samples=4, save_path=f'{LOGS_DIR}/evaluation.png')

if ARCHITECTURE == 'cascade':
    plot_results_from_loader('cascade', test_loader, model, DEVICE, n_samples=4,
                             save_path=f'{LOGS_DIR}/evaluation.png', model0=binary_model)

## Work in progress

In [ ]:
# torch.save(model.state_dict(), CHECKPOINT_DIR + 'model.pth')

checkpoint = torch.load(CHECKPOINT_DIR + 'cascade-model.pth')
model = Unet(3, 1, LAYERS).to(DEVICE)
# model = DualUnet(3, 1, LAYERS).to(DEVICE)
criterion = DiceLoss(1)
model.load_state_dict(checkpoint)

binary_model = Unet(3, 1, LAYERS).to(DEVICE)
checkpoint = torch.load(CHECKPOINT_DIR + 'binary-model.pth')
binary_model.load_state_dict(checkpoint)

## Contour detection method

In [ ]:
from skimage import segmentation
from scipy.ndimage import distance_transform_edt

images, masks = next(iter(val_loader))
images = images.float().to(DEVICE)
masks = masks.long().to(DEVICE)

image = images[0].cpu().numpy().transpose(1, 2, 0) / 255.0
mask = masks[0].cpu().numpy()
prediction = np.zeros_like(mask)
prediction[16:112, 16:112] = 1
prediction[32:96, 32:96] = 2

boundaries = segmentation.find_boundaries(mask, mode='inner').astype(np.uint8)
marked = image.copy()
marked[boundaries == 1] = [0, 0, 0]
dist_map = distance_transform_edt(1 - boundaries)
dist_map = dist_map / dist_map.max()

_, ax = plt.subplots(2, 4, figsize=(15, 7))
ax[0, 0].imshow(mask)
ax[0, 1].imshow(boundaries)
ax[0, 2].imshow(dist_map)
ax[0, 3].imshow(dist_map.max() - dist_map)
ax[1, 0].imshow(prediction)
ax[1, 1].imshow(marked)
plt.show()

In [ ]:
model = Unet(in_channels=3, out_channels=1).to(DEVICE)
loss = DiceLoss(num_classes=1)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(5):
    acc_loss = 0
    for images, masks in val_loader:

        edge_masks = np.zeros((masks.shape[0], masks.shape[1], masks.shape[2]))
        for b in range(masks.shape[0]):
            mask = masks[b].cpu().numpy()
            boundaries = segmentation.find_boundaries(mask, mode='thick').astype(np.uint8)
            edge_masks[b] = boundaries
        masks = torch.from_numpy(edge_masks)

        images = images.float().to(DEVICE)
        masks = masks.long().to(DEVICE)

        outputs = model(images)
        loss_value = loss(outputs, masks)
        acc_loss += loss_value.item()

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    # plot example
    images = images.cpu().numpy()
    masks = masks.cpu().numpy()
    # probs = F.softmax(outputs, dim=1)
    # preds = torch.argmax(probs, dim=1).cpu().numpy()
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).float().cpu().numpy().transpose(0, 2, 3, 1)

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(images[0].transpose(1, 2, 0) / 255.0)
    ax[1].imshow(masks[0])
    ax[2].imshow(preds[0])
    plt.show()

    print(f'Epoch {epoch + 1} loss:', acc_loss / len(val_loader))
